In [1]:
import pandas as pd
import rltk
import re
from difflib import SequenceMatcher
from tqdm import tqdm
import json
import os

In [2]:
os.listdir('../data')

['semantic_joined_dblp.json.zip',
 '.DS_Store',
 'Merged_graphics.csv',
 'semantic_joined_dblp.json',
 'Merged_ai.csv',
 'output.json',
 'semantic_scholar_articles_20000.csv',
 'Merged_vision.csv',
 'Merged_nlp.csv',
 'output_compiled.json']

## Loading Semantic and DBLP JSON

In [34]:
f = open('../data/semantic_joined_dblp.json','r')
t=f.read()
f.close()
js = json.loads(t)
df_js = pd.DataFrame(js)

In [39]:
df_js.head()

,abstract,arxivId,authors,citationVelocity,citations,corpusId,doi,influentialCitationCount,paperId,references,title,topics,url,year,dblp_info
0,The artificial neural networks that are used t...,NaN,"[{'authorId': '1695689', 'name': 'Geoffrey E. ...",150,"[{'arxivId': None, 'authors': [{'authorId': '1...",6138085,10.1007/978-3-642-21735-7_6,52,20f0357688876fa4662f806f985779dce6e24f3c,"[{'arxivId': None, 'authors': [{'authorId': '5...",Transforming Auto-Encoders,"[{'topic': 'Computer vision', 'topicId': '5332...",https://www.semanticscholar.org/paper/20f03576...,2011.0,"{'@score': '3', '@id': '3107620', 'info': {'au..."
1,In order to improve the accuracy of capsule ne...,NaN,"[{'authorId': '144866658', 'name': 'Xin Ning',...",0,[],215723800,10.1109/ACCESS.2020.2982782,0,4c79e754407d41904979746fbc8090545c0aeec9,"[{'arxivId': '1810.10183', 'authors': [{'autho...",BDARS_CapsNet: Bi-Directional Attention Routin...,"[{'topic': 'Routing', 'topicId': '1048', 'url'...",https://www.semanticscholar.org/paper/4c79e754...,2020.0,NaN
2,The current dominant paradigm for feature lear...,1505.01596,"[{'authorId': '33932184', 'name': 'Pulkit Agra...",96,"[{'arxivId': None, 'authors': [{'authorId': '2...",1637703,10.1109/ICCV.2015.13,31,dfbfaaec46d38392f61d683c340ee92a0a66e5d9,"[{'arxivId': None, 'authors': [{'authorId': '2...",Learning to See by Moving,"[{'topic': 'Visual odometry', 'topicId': '644'...",https://www.semanticscholar.org/paper/dfbfaaec...,2015.0,"{'@score': '5', '@id': '2026007', 'info': {'au..."
3,"In this work, we address the problem of improv...",NaN,"[{'authorId': '3458345', 'name': 'Zhun Sun', '...",0,"[{'arxivId': None, 'authors': [{'authorId': '3...",53872389,10.1109/CVPR.2018.00830,2,51bd5966fc992498cb1147d34527e33656c696bb,"[{'arxivId': None, 'authors': [{'authorId': '2...",Feature Quantization for Defending Against Dis...,"[{'topic': 'Distortion', 'topicId': '15080', '...",https://www.semanticscholar.org/paper/51bd5966...,2018.0,"{'@score': '8', '@id': '1024764', 'info': {'au..."
4,"We introduce BSD-GAN, a novel multi-branch and...",1803.08467,"[{'authorId': '39737792', 'name': 'Zili Yi', '...",0,"[{'arxivId': '2009.13311', 'authors': [{'autho...",220666238,10.1109/TIP.2020.3014608,0,70977b9464b94f71c2c4fc68aa03082e5739b6c3,"[{'arxivId': '1710.10196', 'authors': [{'autho...",BSD-GAN: Branched Generative Adversarial Netwo...,"[{'topic': 'BSD', 'topicId': '10760', 'url': '...",https://www.semanticscholar.org/paper/70977b94...,2020.0,"{'@score': '15', '@id': '211133', 'info': {'au..."


In [35]:
ls = [i for i in os.listdir('../data') if "Merged" in i]

In [36]:
df = pd.DataFrame()
for i in ls:
    df_2 = pd.read_csv('../data/'+i)
    df = df.append(df_2)

In [41]:
df.head()

,ID,title,authors,published,updated,abstract,categories,citations,arxiv_url,gscholar_url,journal
0,0,Dynamic Graph CNN for Learning on Point Clouds,"['Yue Wang', 'Sanjay E. Sarma', 'Justin M. Sol...",2018-01-24T01:14:04Z,2019-06-11T06:11:21Z,Point clouds provide a flexible geometric repr...,['cs.CV'],536.0,http://arxiv.org/abs/1801.07829v2,http://scholar.google.com/scholar?oi=bibs&clus...,"ACM Transactions on Graphics (TOG) 38 (5), 1-12"
1,1,Looking to Listen at the Cocktail Party: A Spe...,"['Avinatan Hassidim', 'Oran Lang', 'Michael Ru...",2018-04-10T16:28:59Z,2018-08-09T21:22:37Z,We present a joint audio-visual model for isol...,"['cs.SD', 'cs.CV', 'eess.AS']",189.0,http://arxiv.org/abs/1804.03619v2,http://scholar.google.com/scholar?oi=bibs&clus...,"ACM Transactions on Graphics (TOG) 37 (4), 1-11"
2,2,Convergence of univariate non-stationary subdi...,"['Carla Manni', 'Costanza Conti', 'Marie-Laure...",2014-10-10T10:30:14Z,2014-10-10T10:30:14Z,A new equivalence notion between non-stationar...,['math.NA'],34.0,http://arxiv.org/abs/1410.2729v1,http://scholar.google.com/scholar?oi=bibs&clus...,"Computer Aided Geometric Design 37, 1-8"
3,3,HDR image reconstruction from a single exposur...,"['Joel Kronander', 'Rafał K. Mantiuk', 'Gyorgy...",2017-10-20T10:48:22Z,2017-10-20T10:48:22Z,Camera sensors can only capture a limited rang...,"['cs.CV', 'cs.GR', 'cs.LG']",145.0,http://arxiv.org/abs/1710.07480v1,http://scholar.google.com/scholar?oi=bibs&clus...,"ACM Transactions on Graphics (TOG) 36 (6), 1-15"
4,4,Learning Style Similarity for Searching Infogr...,"['Aaron Hertzmann', 'Mira Dontcheva', 'Babak S...",2015-05-05T22:59:32Z,2015-05-05T22:59:32Z,Infographics are complex graphic designs integ...,"['cs.GR', 'cs.CV', 'cs.HC', 'cs.IR', 'cs.MM']",24.0,http://arxiv.org/abs/1505.01214v1,http://scholar.google.com/scholar?oi=bibs&clus...,Proceedings of the 41st Graphics Interface Con...


## Load Data

### Naïve Data Merge

In [42]:
# How many matches can be found with a naÏve identical string approach?
d1 = df_js['title'].values
d2 = df['title'].values
print(len([1 for w in d1 if w in d2]))

159


## RLTK Data Merge

In [43]:
# RLTK Tokenizer
tokenizer = rltk.CrfTokenizer()

**Arxiv/Google Scholar Dataset**

In [52]:
df_js.reset_index(inplace = True)

In [54]:
df_js.keys()

Index(['index', 'abstract', 'arxivId', 'authors', 'citationVelocity',
       'citations', 'corpusId', 'doi', 'influentialCitationCount', 'paperId',
       'references', 'title', 'topics', 'url', 'year', 'dblp_info'],
      dtype='object')

In [62]:
df.reset_index(inplace=True)

In [63]:
df.keys()

Index(['index', 'ID', 'title', 'authors', 'published', 'updated', 'abstract',
       'categories', 'citations', 'arxiv_url', 'gscholar_url', 'journal'],
      dtype='object')

In [90]:
import datetime

In [141]:
df_js['authors'][0]

"[{'authorId': '1695689', 'name': 'Geoffrey E. Hinton', 'url': 'https://www.semanticscholar.org/author/1695689'}, {'authorId': '2064160', 'name': 'A. Krizhevsky', 'url': 'https://www.semanticscholar.org/author/2064160'}, {'authorId': '49185042', 'name': 'S. Wang', 'url': 'https://www.semanticscholar.org/author/49185042'}]"

In [144]:
df['authors'][0]

"['Yue Wang', 'Sanjay E. Sarma', 'Justin M. Solomon', 'Yongbin Sun', 'Ziwei Liu', 'Michael M. Bronstein']"

In [152]:
import ast

In [127]:
def getPlainString(string):
    return ''.join([s for s in string if not s.isnumeric()]).strip()


{'Hinton', 'Krizhevsky', 'Wang'}

In [148]:
class ArxivRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = 'ArxivRecord'
        
    @property
    def id(self):
        return str(self.raw_object['ID'])
    
    @rltk.cached_property
    def authors_string(self):
        return self.raw_object['authors']
    
    @rltk.cached_property
    def title_string(self):
        return self.raw_object['title']
        
#     @rltk.cached_property
#     def summary_string(self):
#         return self.raw_object['summary']
    
    @rltk.cached_property
    def categories_string(self):
        return self.raw_object['categories']
    
    @rltk.cached_property
    def published_string(self):
        return self.raw_object['published']
    
    @rltk.cached_property
    def updated_string(self):
        return self.raw_object['updated']
        
    @rltk.cached_property
    def blocking_author_tokens(self):
        return set([getPlainString(s).split(' ')[-1] for s in ast.literal_eval(self.raw_object['authors'])])
    
#     @rltk.cached_property
#     def url_string(self):
#         return self.raw_object['url']
    
    @rltk.cached_property
    def blocking_tokens(self):
        tokens = ' '.join([self.title_string])
        tokens = re.sub(r'\bThe\b', '', tokens)
        tokens = re.sub(r'\bthe\b', '', tokens)
        tokens = re.sub(r'\bof\b', '', tokens)
        tokens = re.sub(r"\b's\b", '', tokens)
        tokens = re.sub(r'\band\b', '', tokens)
        tokens = re.sub(r'\bI\b', '', tokens)
        tokens = re.sub(r'\bA\b', '', tokens)
        tokens = re.sub(r'\bin\b', '', tokens)
        tokens = re.sub(r'\bfor\b', '', tokens)
        tokens = re.sub(r'\bon\b', '', tokens)
        tokens = re.sub(r'\bwith\b', '', tokens)
        return set(tokenizer.tokenize(tokens))

In [149]:
ds_1 = rltk.Dataset(reader=rltk.DataFrameReader(df), record_class=ArxivRecord, adapter=rltk.MemoryKeyValueAdapter())
print(type(ds_1))
ds_1.generate_dataframe().head(3)

<class 'rltk.dataset.Dataset'>


,id,authors_string,title_string,categories_string,published_string,updated_string,blocking_author_tokens,blocking_tokens
0,0,"['Barret Zoph', 'Kevin Knight']",Multi-Source Neural Translation,['cs.CL'],2016-01-05T00:49:22Z,2016-01-05T00:49:22Z,"{Zoph, Knight}","{Source, -, Neural, Multi, Translation}"
1,1,"['Xiaodong He', 'Asli Celikyilmaz', 'Antoine B...",Deep Communicating Agents for Abstractive Summ...,['cs.CL'],2018-03-27T23:29:23Z,2018-08-15T18:54:22Z,"{Choi, Bosselut, Celikyilmaz, He}","{Agents, Communicating, Deep, Summarization, A..."
2,2,"['Furu Wei', 'Ming Zhou', 'Nan Yang', 'Qingyu ...",Selective Encoding for Abstractive Sentence Su...,['cs.CL'],2017-04-24T07:57:37Z,2017-04-24T07:57:37Z,"{Yang, Zhou, Wei}","{Selective, Encoding, Sentence, Summarization,..."


**Google Scholar Dataset**

In [150]:
class SemanticRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = 'SemanticRecord'
        
    @property
    def id(self):
        return str(self.raw_object['index'])
    
    @rltk.cached_property
    def authors_string(self):
        return self.raw_object['authors']
    
    @rltk.cached_property
    def title_string(self):
        return self.raw_object['title']
        
#     @rltk.cached_property
#     def journal_string(self):
#         return self.raw_object['journal']
    
    @rltk.cached_property
    def citations_string(self):
        return self.raw_object['citations']
    
    @rltk.cached_property
    def year_string(self):
        return self.raw_object['year']
    
    @rltk.cached_property
    def url_string(self):
        return self.raw_object['url']
    
    
    @rltk.cached_property
    def blocking_author_tokens(self):
        return set([getPlainString(s['name']).split(' ')[-1] for s in ast.literal_eval((self.raw_object['authors']))])

    
    @rltk.cached_property
    def blocking_tokens(self):
        tokens = ' '.join([self.title_string])
        tokens = re.sub(r'\bThe\b', '', tokens)
        tokens = re.sub(r'\bthe\b', '', tokens)
        tokens = re.sub(r'\bof\b', '', tokens)
        tokens = re.sub(r"\b's\b", '', tokens)
        tokens = re.sub(r'\band\b', '', tokens)
        tokens = re.sub(r'\bI\b', '', tokens)
        tokens = re.sub(r'\bA\b', '', tokens)
        tokens = re.sub(r'\bin\b', '', tokens)
        tokens = re.sub(r'\bfor\b', '', tokens)
        tokens = re.sub(r'\bon\b', '', tokens)
        tokens = re.sub(r'\bwith\b', '', tokens)
        return set(tokenizer.tokenize(tokens))

In [151]:
ds_2 = rltk.Dataset(reader=rltk.DataFrameReader(df_js), record_class=SemanticRecord, adapter=rltk.MemoryKeyValueAdapter())
print(type(ds_2))
ds_2.generate_dataframe().head(3)

<class 'rltk.dataset.Dataset'>


,id,authors_string,title_string,citations_string,year_string,url_string,blocking_author_tokens,blocking_tokens
0,0,"[{'authorId': '1695689', 'name': 'Geoffrey E. ...",Transforming Auto-Encoders,"[{'arxivId': None, 'authors': [{'authorId': '1...",2011.0,https://www.semanticscholar.org/paper/20f03576...,"{Krizhevsky, Wang, Hinton}","{Auto, Encoders, -, Transforming}"
1,1,"[{'authorId': '144866658', 'name': 'Xin Ning',...",BDARS_CapsNet: Bi-Directional Attention Routin...,[],2020.0,https://www.semanticscholar.org/paper/4c79e754...,"{Nie, Ning, Sun, Chen, Tian, Li, Lu}","{Directional, _, -, Attention, Sausage, Capsul..."
2,2,"[{'authorId': '33932184', 'name': 'Pulkit Agra...",Learning to See by Moving,"[{'arxivId': None, 'authors': [{'authorId': '2...",2015.0,https://www.semanticscholar.org/paper/dfbfaaec...,"{Agrawal, Malik, Carreira}","{Moving, Learning, by, See, to}"


set

### Blocking

In [153]:
# Generate blocks from tokens
token_blocker = rltk.TokenBlockGenerator()
blocks = token_blocker.generate(
    token_blocker.block(ds_1, property_='blocking_author_tokens'),
    token_blocker.block(ds_2, property_='blocking_author_tokens'))
print(type(blocks))

<class 'rltk.blocking.block.Block'>


In [154]:
# Extract all record pairs from the block
record_pairs = rltk.get_record_pairs(ds_1, ds_2, block=blocks)

# Get the total number of record pairs generated
compared_pairs = len(list(record_pairs))

# Get the number of elements in each rltk.Dataset
tally_imdb = ds_1.generate_dataframe().shape[0]
tally_tmd = ds_2.generate_dataframe().shape[0]

# Calculate the total number of pairs if both datasets were to be compared without any blocking (eg: a double for loop)
tally_unblocked = tally_imdb * tally_tmd

# Calculate how much smaller the blocked pairings are
reduction_ratio = compared_pairs / tally_unblocked

# Calculate the reduction ratio (the inverse of the )
reduction_ratio = 1 - reduction_ratio
print(f'Reduction Ratio: {reduction_ratio:.5f}')

Reduction Ratio: 0.93040


In [180]:
compared_pairs

3036085

### Matching Functions

In [172]:
def title_similarity(arxiv_tuple, gscholar_tuple):
    arxiv_title = arxiv_tuple.title_string.strip().lower()
    gscholar_title = gscholar_tuple.title_string.strip().lower()
    similarity = SequenceMatcher(None, arxiv_title, gscholar_title).ratio()

    penalties = sum([len(arxiv_title)<=6,
                     len(gscholar_title)<=6])

    return similarity * (0.9**penalties)

In [173]:
def author_similarity(arxiv_tuple, gscholar_tuple):
    arxiv_author = ' '.join(arxiv_tuple.authors_string).strip().lower()
    gscholar_author = ' '.join(gscholar_tuple.authors_string).strip().lower()
    similarity = SequenceMatcher(None, arxiv_author, gscholar_author).ratio() 
    return similarity

In [177]:
def year_similarity(arxiv_tuple, gscholar_tuple):
#     arxiv_year = int(float(arxiv_tuple.updated_string[0:4]))
#     print(arxiv_tuple.published_string)
    if str(arxiv_tuple.published_string) == "nan":
        return 0
    arxiv_year = datetime.datetime.strptime(arxiv_tuple.published_string, '%Y-%m-%dT%H:%M:%SZ').year

    gscholar_year = int(float(gscholar_tuple.year_string))
    similarity = 1 /(1 + abs(arxiv_year-gscholar_year))
    return similarity

In [178]:
def elementwise_similarity(arxiv_tuple, gscholar_tuple, match_threshold=0.75):
    sim_title = title_similarity(arxiv_tuple, gscholar_tuple)
    sim_author = author_similarity(arxiv_tuple, gscholar_tuple)
    sim_year = year_similarity(arxiv_tuple, gscholar_tuple)

    element_similarity = (0.70 * sim_title) + (0.15 * sim_author) + (0.15 * sim_year)

    return element_similarity > match_threshold, element_similarity

In [ ]:
# Predict matches for all pairs in the blocked data 
print(f'Arxiv samples: {df.shape[0]}')
print(f'Semantic samples: {df_js.shape[0]}')

summary_df = pd.DataFrame()
THRESHOLDS = [T/100 for T in range(65, 101, 5)]
c=0
# Iterate through various thresholds to find the most matches without any duplicates
for T in tqdm(THRESHOLDS):

    # Set to store pairs of IDs matched
    ids_matched = set()
    
    # Iterate through candidates on the block
    for block_id, arxiv_id, gscholar_id in blocks.pairwise(ds_1, ds_2):
        
        # Find similarity at a given threshold
        match , similarity = elementwise_similarity(ds_1.get_record(arxiv_id),
                                                    ds_2.get_record(gscholar_id),
                                                    match_threshold=T)
        # If a match is found, add to the set of matches
        if match:
            ids_matched.add((arxiv_id, gscholar_id))
        c=c+1
        if c>1000:
            break
    
    # Count the number of unique elements derived from each source
    set_a = set()
    set_b = set()
    for tp in ids_matched:
        set_a.add(tp[0])
        set_b.add(tp[1])
    
    summary_df.at[T, 'Matches'] = int(len(ids_matched))
    summary_df.at[T, 'Set_A Size'] = int(len(set_a))
    summary_df.at[T, 'Set_B Size'] = int(len(ids_matched))
    summary_df.at[T, 'Duplicates'] = int((len(ids_matched)-len(set_a)) + (len(ids_matched)-len(set_b)))
    
summary_df

In [193]:
len(ids_matched)

2700133

In [ ]:
# Find the lowest threshold which gives no duplicates
optimal_threshold = summary_df[summary_df['Duplicates']==0].index[0]
optimal_threshold

In [ ]:
# Manually set threshold to 0.85 to trade 104 extra True Positives for 3 extra False Positive
# optimal_threshold = 0.85

In [ ]:
# Generate matches based on the optimal (no-duplicate) threshold
print(f'Arxiv samples: {df_arxiv.shape[0]}')
print(f'GScholar samples: {df_gscholar.shape[0]}')

# Store tuples of matches IDs, as well as singletons witouth a match
ids_matched = set()
singles_arxiv = set()
singles_gscholar = set()

# Write matches (and non-matches) to a CSV
with open(f'Matches_{TOPIC}.csv', 'w') as predictions_full:
    for block_id, arxiv_id, gscholar_id in blocks.pairwise(ds_arxiv, ds_gscholar):

        match , similarity = elementwise_similarity(ds_arxiv.get_record(arxiv_id),
                                                    ds_gscholar.get_record(gscholar_id),
                                                    match_threshold=optimal_threshold)

        if match:
            ids_matched.add((arxiv_id, gscholar_id))
        else:
            singles_arxiv.add(arxiv_id)
            singles_gscholar.add(gscholar_id)
    
    # After finding all matches, write them to a csv
    for match_pair in ids_matched:
        predictions_full.write(f'{match_pair[0]},{match_pair[1]},1\n')
        # And ensure that no item in the matches is counted as a single
        try:
            singles_arxiv.remove(match_pair[0])
        except:
            pass
        try:
            singles_gscholar.remove(match_pair[1])
        except:
            pass
    
    # Then write all the singles which didn't find a match
    NULL = None
    for arxiv_id in singles_arxiv:
        predictions_full.write(f'{arxiv_id},{NULL},0\n')
    for gscholar_id in singles_gscholar:
        predictions_full.write(f'{NULL},{gscholar_id},0\n')        
        
print()
print(f'Matches: {len(ids_matched)}')
print(f'Non-Matches Arxiv: {len(singles_arxiv)}')
print(f'Non-Matches GScholar: {len(singles_gscholar)}')

### Create Merged Dataset

In [ ]:
import rdflib
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD

MYNS = Namespace('http://inf558.org/myfakenamespace#')
SCHEMA = Namespace("https://schema.org/")

# Initliaze the graph
g = rdflib.Graph()

# Bind namespace and prefixes
g.bind('my_ns', MYNS)
g.bind('schema', SCHEMA)
g.bind('rdf', RDF)
g.bind('rdfs', RDFS)
g.bind('xsd', XSD)

In [ ]:
# Load predictions to be used in populating the RDF
predictions_df = pd.read_csv(f'Matches_{TOPIC}.csv', header=None, names=['ARXIV_ID', 'GSCHOLAR_ID', 'LABEL'])
print(f'predictions_df.shape: {predictions_df.shape}')
predicted_matches = predictions_df['LABEL'].sum()
print(f'predicted matches: {predicted_matches}  [{100*predicted_matches/predictions_df.shape[0]:.2f} %]')
predictions_df.head()

In [ ]:
# Dataframe to store the merged datasets
df_merged = pd.DataFrame(columns = ['ID', 'title', 'authors', 'published', 'updated', 
                                    'abstract', 'categories', 'citations', 'arxiv_url', 'gscholar_url'],
                         dtype='object')
json_merged = {}

NEW_ID = 0

# Populate the RDF with predictions with a positive (1) label
for idx, row in tqdm(predictions_df.iterrows(), total=predictions_df.shape[0]):
    
    # Populate the json object
    json_merged[NEW_ID] = {}
    
    ### URI ###
    node_uri = URIRef(str(NEW_ID))
    g.add((node_uri, RDF.type, SCHEMA.ScholarlyArticle))
    df_merged.at[NEW_ID, 'ID'] = NEW_ID
    json_merged[NEW_ID]['ID'] = NEW_ID

    
    ### Title ###
    try:
        title_arxiv = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['title'].values[0])
    except:
        title_arxiv = '<___>'
    try:
        title_gscholar = str(df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['title'].values[0])
    except:
        title_gscholar = '<___>'
    title = title_arxiv if title_arxiv != '<___>' else title_gscholar if title_gscholar != '<___>' else None
    g.add((node_uri, SCHEMA.headline, Literal(title, datatype=SCHEMA.Text)))
    df_merged.at[NEW_ID, 'title'] = title
    json_merged[NEW_ID]['title'] = title

    
    ### Author(s) ###
    try:
        author_arxiv = df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['authors'].values[0]
        author_arxiv = [name.strip() for name in author_arxiv if name != '<___>']
    except:
        author_arxiv = '<___>'
    try:
        author_gscholar = df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['authors'].values[0]
        author_gscholar = [name.strip() for name in author_gscholar if name != '<___>']
    except:
        author_gscholar = '<___>'
    if author_arxiv != '<___>':
        authors = list(set(author_arxiv))
    else:
        authors = list(set(author_gscholar))           
    [g.add((node_uri, SCHEMA.author, Literal(author, datatype=SCHEMA.Person))) for author in authors]
    df_merged.at[NEW_ID, 'authors'] = authors
    json_merged[NEW_ID]['authors'] = authors
                       
    ### Published ###
    try:
        published = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['published'].values[0])
        g.add((node_uri, SCHEMA.datePublished, Literal(published, datatype=SCHEMA.DateTime)))
        df_merged.at[NEW_ID, 'published'] = published
        json_merged[NEW_ID]['published'] = published
    except:
        pass
        
                       
    ### Updated ###
    try:
        updated = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['updated'].values[0])
        g.add((node_uri, SCHEMA.dateModified, Literal(updated, datatype=SCHEMA.DateTime)))
        df_merged.at[NEW_ID, 'updated'] = updated
        json_merged[NEW_ID]['updated'] = updated
    except:
        pass
          
                       
    ### Abstract ###
    try:
        abstract = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['summary'].values[0]).strip()
        g.add((node_uri, SCHEMA.abstract, Literal(abstract, datatype=SCHEMA.Text)))
        df_merged.at[NEW_ID, 'abstract'] = abstract
        json_merged[NEW_ID]['abstract'] = abstract
    except:
        pass
       
                       
    ### Categories ###
    try:
        categories = df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['categories'].values[0]
        categories = [name.strip() for name in categories if name != '<___>']
        [g.add((node_uri, SCHEMA.genre, Literal(category, datatype=SCHEMA.Text))) for category in categories]
        df_merged.at[NEW_ID, 'categories'] = categories
        json_merged[NEW_ID]['categories'] = categories
    except:
        pass
          
                       
    ### Journal ###
    try:
        journal = str(df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['journal'].values[0])
        g.add((node_uri, SCHEMA.publisher, Literal(journal, datatype=SCHEMA.Periodical))) #datatype=SCHEMA.Organisation
        df_merged.at[NEW_ID, 'journal'] = journal
        json_merged[NEW_ID]['journal'] = journal
    except:
        pass
     
                       
    ### Citations ###
    try:
        citations = str(df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['citations'].values[0])
        g.add((node_uri, SCHEMA.commentCount, Literal(citations, datatype=SCHEMA.Integer)))
        df_merged.at[NEW_ID, 'citations'] = citations
        json_merged[NEW_ID]['citations'] = citations
    except:
        pass
            
                       
    ### Arxiv URL ###
    try:
        arxiv_url = str(df_arxiv[df_arxiv['ID'] == str(row['ARXIV_ID'])]['url'].values[0])
        g.add((node_uri, SCHEMA.url, Literal(arxiv_url, datatype=SCHEMA.URL)))
        df_merged.at[NEW_ID, 'arxiv_url'] = arxiv_url    
        json_merged[NEW_ID]['arxiv_url'] = arxiv_url
    except:
        pass
        
                       
    ### Google Scholar URL ###
    try:
        gscholar_url = str(df_gscholar[df_gscholar['ID'] == str(row['GSCHOLAR_ID'])]['url'].values[0])
        g.add((node_uri, SCHEMA.url, Literal(gscholar_url, datatype=SCHEMA.URL)))
        df_merged.at[NEW_ID, 'gscholar_url'] = gscholar_url   
        json_merged[NEW_ID]['gscholar_url'] = gscholar_url
    except:
        pass
             
                       
    NEW_ID += 1
    
# Save to disk using turtle format
g.serialize(f'Triples_{TOPIC}.ttl.', format="turtle")

# And save the merged DataFrame as CSV
df_merged.to_csv(f'Merged_{TOPIC}.csv', index=False)

# Also save as Json, just because
with open(f'Json_{TOPIC}.json', 'w') as fout:
    json.dump(json_merged, fout)

In [ ]:
df_merged.head(3)

In [ ]:
df_merged.info()

In [ ]:
json_merged

## End